In [1]:
## Preamble: Package Loading
import numpy as np
from sklearn import linear_model
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pandas as pd
import itertools as iter
import os
import datetime as dt
import json
# Preamble working directory retreival
wkng_folder = os.getcwd()

In [2]:
input_filename = 'pscdata_6_19_1923.json'
data_file = '/Users/ericpenner/Google_Drive/Research/pan_sel_cntrl/data'
input_file_full = ''.join([data_file,'/',input_filename])
with open(input_file_full) as f_obj: 
    pscdata = json.load(f_obj)
# Initializing the data sets metadata dictionary 
inpt = pscdata[0][0].copy()

k=10
data_err = pd.DataFrame(pscdata[k][0]['err_df'], columns = pscdata[0][1]['Derr_nms'])  
data_inst = pd.DataFrame(pscdata[k][0]['inst_df'], columns = pscdata[0][1]['Dins_nms'])
data_pan = pd.DataFrame(pscdata[k][0]['prim_df'], columns = pscdata[0][1]['Dlng_nms'])

In [3]:
inpt

{'c_inst': 3,
 'cin': 'crs',
 'dep_nm': 'Y',
 'en_nm': ['Z1,1', 'Z1,2'],
 'err_vpro': [0.8, 0.36],
 'ex_nm': ['Z2,1', 'Z2,2'],
 'ex_vpro': [0.5],
 'frc': 0,
 'inst_vpro': [0.5, 0.25, -0.1],
 'n_end': 2,
 'n_exo': 2,
 'ncs': 10,
 'nds': 1000,
 'ntp': 30,
 'r_seed': 19977,
 'sec_pan': 1,
 't_inst': 10,
 'tin': 't'}

In [4]:
pd.DataFrame(pscdata[0][1]['coeff'][1])

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,0.0,1.0
1,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,1.0
2,1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,0.0,0.0
3,1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0
4,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,1.0
5,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,0.0
6,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,1.0
7,1.0,-1.0,0.0,0.0,-1.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0
8,1.0,-1.0,0.0,1.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0
9,1.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0


<h2>Estimator With Only ScikitLearn Lasso</h2>

In [7]:
dp = data_pan
di = data_inst
de = data_err

dep_nm = inpt['en_nm']
ex_nm = inpt['ex_nm']
n_exo = inpt['n_exo']
ins_nm = list(di.columns)[1:]
adep_nm = [ 0 for i in range(len(dep_nm))]
epsil = 0.05
ain_cf = [ 0 for i in range(len(dep_nm))]
ain_cf_rm = [ 0 for i in range(len(dep_nm))]
excf1 = []

for g in range(len(ex_nm)):
    excf1 = []
    for k in range(inpt['ncs']):
        dsl = dp.loc[dp['crs']==k+1,['t'] + [dep_nm[g]] + ex_nm]
        dsl = pd.merge(dsl,di,how ="inner", on='t')
        lin_reg = linear_model.LinearRegression()
        lin_reg.fit(dsl.loc[:,ex_nm+ins_nm].values,dsl.loc[:,dep_nm[g]].values.reshape(dsl.shape[0],1))
        excf1.append([lin_reg.coef_[0][i] for i in range(inpt['n_exo'])])

    incf = [0 for i in range(inpt['t_inst'])]    
    excf = [np.mean([excf1[i][j] for i in range(len(excf1))] ) for j in range(n_exo)]

    adep_nm[g] = ''.join(['a',dep_nm[g]])
    dp[adep_nm[g]] = (dp.loc[:,dep_nm[g]].values.reshape(dp.shape[0],1)
                     -dp.loc[:,ex_nm].values.dot(np.array(excf).reshape(inpt['n_exo'],1)))
    
    ain_cf1= []
    for k in range(inpt['ncs']):
        ds2 = dp.loc[dp['crs']==k+1,['t'] + [adep_nm[g]] ]
        ds2 = pd.merge(ds2,data_inst,how ="inner", on='t')
        lasso_reg = linear_model.Lasso(alpha = 0.4)
        lasso_reg.fit(ds2.loc[:,ins_nm].values,ds2.loc[:,adep_nm[g]].values.reshape(ds2.shape[0],1))
        ain_cf1.append(list(lasso_reg.coef_))

    ain_cf[g] = excf
    for j in  range(len(ain_cf1[0])): 
        a = [ain_cf1[i][j] for i in range(len(ain_cf1)) if np.abs(ain_cf1[i][j]) > epsil]
        if not a:
            ain_cf[g].append(0)
        else:
            ain_cf[g].append(np.mean(a))    

    ain_cf_rm[g] = [[1,1] + [int(np.abs(ain_cf1[j][i])>epsil) 
                            for i in range(len(ain_cf1[0]))] 
                            for j in range(len(ain_cf1))]

In [8]:
ain_cf

[[-0.93202104394930241,
  0.99717848004354437,
  0.046272446324877342,
  -1.0393972950714356,
  0.51670905604036954,
  0.87426053105341506,
  0.32360718500682439,
  0.42744855947047955,
  -0.074723470538190229,
  0.46660589161111482,
  -1.0913422830285153,
  -0.52851703338989531],
 [1.0589440008103959,
  -1.016310366546052,
  0,
  0,
  -0.29748567649101382,
  0.4493652219265028,
  -0.21125698029242695,
  -0.60413138380062625,
  -0.40432019023250793,
  -0.48878075927309284,
  0.26095064953119618,
  0.69683156079169828]]

In [9]:
pd.DataFrame(ain_cf1)

,0,1,2,3,4,5,6,7,8,9
0,-0.00000,0.0000,0.000000,0.449365,0.208153,-0.000000,-0.000000,-0.000000,0.000000,0.306967
1,-0.00000,0.0000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.532747,0.755600
2,0.00000,0.0000,-0.000000,-0.000000,-0.630667,-1.162640,-0.212077,-0.000000,-0.000000,0.000000
3,-0.00000,-0.0000,-0.000000,-0.000000,0.000000,-0.209925,-0.000000,-0.376849,-0.000000,0.000000
4,0.00000,0.0000,0.000000,0.000000,-0.000000,-0.081423,-0.179932,0.000000,0.316568,1.225309
5,-0.00000,-0.0000,0.000000,-0.000000,-0.000000,-0.687390,-0.824936,-0.915903,-0.000000,0.000000
6,0.00000,0.0000,0.000000,-0.000000,-0.000000,-1.077360,-0.096513,-0.000000,-0.000000,0.499450
7,-0.00000,-0.0000,-0.131372,0.000000,-0.000000,-0.062002,-0.000000,0.000000,0.000000,0.000000
8,0.00000,0.0106,0.000000,-0.000000,-0.000000,-0.948180,-0.107136,-0.173590,-0.000000,-0.000000
9,0.00551,0.0000,-0.463600,-0.024132,-0.000000,-0.000000,-1.005327,-0.000000,-0.066463,-0.000000


In [10]:
excf1

[[1.0745527437101305, -1.1940733380049864],
 [0.89664807580319705, -0.6990502598348125],
 [0.6463010838920159, -0.39919114519071963],
 [0.91656488270119496, -0.61197115506219191],
 [1.2809464651645102, -1.1332974118232455],
 [1.472919410131289, -1.5620606858156183],
 [1.0546259384140311, -1.0724402222062905],
 [0.99651029680804615, -1.1235018249926139],
 [1.1092580764938462, -1.1379567778883093],
 [1.1411130349856973, -1.2295608446417332]]